In [ ]:
def number_of_columns():
    total = 0
    for key in columns_to_vectorize:
        total += columns_to_vectorize[key]
    return total

columns_to_vectorize = {
    'Title': 500,
    'FullDescription': 3000,
    'LocationRaw': 100,
    'LocationNormalized': 100,
    'ContractType': 2,
    'ContractTime': 2,
    'Company': 500,
    'Category': 20,
    'SourceName': 100
}

No dicionário acima, descreve-se quais colunas do CSV de treino serão utilizados mais abaixo no código, bem como seu peso para o CSV de saída. O método de número de colunas apenas retorna a soma de todos os elemtos da direita do dicionário.

In [ ]:
NUMBER_OF_ROWS = 50000

new_file = open("Vectorized.csv","w")
writer = csv.writer(new_file, delimiter=',')

with zipfile.ZipFile('Train_rev1.zip') as myzip:
    with myzip.open('Train_rev1.csv') as myfile:
        df = pd.read_csv(myfile, nrows=NUMBER_OF_ROWS)

definite_row = []
definite_row.append([str(x) for x in range(0, number_of_columns())])
definite_row[0].append('Salary')

A variável NUMBER_OF_ROWS diz o tamanho do CSV de saída, após os dados serem tratados.

O próximo passo é abrir O CSV de origem, que está contido em um ZIP chamado Train_rev1.zip.

Então, é criado o elemento definite_row, que é uma lista de listas, em que cada lista interna é uma abstração para uma linha do CSV de saída.

As próximas operações sobre o definite_row servem para criar uma linha de cabeçalho pro CSV de saída em que a última coluna chama-se "Salary". As colunas anteriores são nomeadas por um número de 0 ao número de colunas.

Note também que nesse passo é criado o CSV de saída, que chama-se "Vectorized.csv" no módo de escrita.

In [ ]:
for i in range(0, NUMBER_OF_ROWS):
    definite_row.append([])

writer.writerow(definite_row[0])


O primeiro loop, que vai de 0 ao número de linhas, adiciona uma lista vazia a cada iteração.

Se cada lista dentro de definite_row é uma linha do CSV, o que esse passo faz nada mais é do que inicializar uma linha que será preenchida depois.

Ao mesmo tempo, a linha de cabeçalho já é escrita no CSV de saída.

In [ ]:
for key in columns_to_vectorize:
    count_vectorizer = CountVectorizer(max_features=columns_to_vectorize[key], stop_words='english')
    transformed = count_vectorizer.fit_transform(df[key].values.astype('U')).toarray()
    for index, value in enumerate(transformed):
        value = value.tolist()
        definite_row[index+1].extend(value)

for i in range(0, NUMBER_OF_ROWS):
    definite_row[i+1].extend([str(df['SalaryNormalized'][i])])
    writer.writerow(definite_row[i+1])

new_file.close()

Esse passo é a cereja do bolo.

O CountVectorizer é uma classe que serve para tratar inputs que são textos. Ele pega um conjunto de strings e aprende delas o dicionário possível de palavras. A lista de strings passada são as colunas do CSV de entrada.

O argumento max_features descreve o número máximo de palavras que será aprendida. No caso, esse número é descrito pelo dicionário supracitado. A coluna 'Title', por exemplo, pode ter um dicionário de 500 palavras diferentes.

O argumento stop_words, que recebe o valor 'english', serve para retirar palavras desnecessárias nas strings. Entre essas palavras, entram preposições e outras coisas não descritivas.

O método fit_transform, que recebe a coluna, efetivamente aprenderá dela as palavras enquanto retorna uma lista de listas, em que cada uma é a frequência do dicionário aprendido para cada linha da entrada.

Após termos essa informação, convertemos ela em uma lista do Python e a adicionamos no nosso elemento de definite_row. O extend faz apenas com que as listas sejam adicionadas como elementos de mesmo nível, ou seja, não criar outra lista de listas.

O próximo loop serve para pegar cada linha de informação adquirida no loop anterior e escrevê-la no CSV de saída.